<h1> Adding the coordinates of each neighbourhood</h1>

We start by creating the dataset from the neighbourhoods with the previous structure

In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
table = html_soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')
table_columns = table.find_all('th')
type(table_columns)
#Get columns name
c = []
for th in table_columns:
    text = th.text.replace('\n', '')
    c.append(text)
    
#Get rows
r = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if(row == []):
        continue
    row = [element.strip('\n') for element in row]
    r.append(row)

df = pd.DataFrame(r, columns=c)
df = df[df['Borough'] != 'Not assigned']
for i,row in df.iterrows():
    if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']
df_aux = df[['Postcode', 'Borough', 'Neighbourhood']]
df_aux = df_aux.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index()

df = pd.merge(df, df_aux, on=['Postcode'], how='inner')
df = df[['Postcode','Borough','Neighbourhood_y']]
df=df.rename(columns = {'Neighbourhood_y':'Neighbourhood'})
df[:5]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


Get the coordinates from the csv, geocoder was not working

In [4]:
geo_data = pd.read_csv("https://cocl.us/Geospatial_data") 

Add to each neighbourhood its coordinates

In [5]:
geo_data=geo_data.rename(columns = {'Postal Code':'Postcode'})
df = pd.merge(df, geo_data, on=['Postcode'], how='inner')

Display the first 5 rows

In [6]:
df[:5]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
